<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependancies


Install the required dependencies for making a custom model.

In [ ]:
# Import Tensorflow 2.0
import tensorflow as tf 
import numpy as np
import cv2 as cv
import glob
import random

# Check that we are using a GPU, if not switch runtimes in Google Collab
#   using Runtime > Change Runtime Type > GPU
# Else if on local system without GPU ignore this error or comment out the below line
assert len(tf.config.list_physical_devices('GPU')) > 0

# Create Dataset

Create a dataset from the included images in the [dataset](dataset) folder.

In [ ]:
def createDataset():
    human_paths = glob.glob('./dataset/human/*.jpg')
    no_human_paths = glob.glob('./dataset/no_human/*.jpg')
    if len(human_paths) == 0 or len(no_human_paths) == 0:
        raise Exception('Dataset could not be found!')
    human_images = [np.split(cv.imread(path), 3, axis=2)[0] for path in human_paths]
    no_human_images = [np.split(cv.imread(path), 3, axis=2)[0] for path in no_human_paths]
    random.shuffle(human_images)
    random.shuffle(no_human_images)
    half_size = min(len(human_images), len(no_human_images))
    dataset = list(zip(human_images[:half_size], np.ones(half_size))) + list(zip(no_human_images[:half_size], np.zeros(half_size)))
    random.shuffle(dataset)
    split_size = int(len(dataset) * 0.8)
    train_dataset = dataset[:split_size]
    test_dataset = dataset[split_size:]
    train_images, train_labels = zip(*train_dataset)
    test_images, test_labels = zip(*test_dataset)
    train_images, test_images = np.array(train_images), np.array(test_images)
    train_labels, test_labels = np.array(train_labels), np.array(test_labels)
    return train_images, train_labels, test_images, test_labels

train_images, train_labels, test_images, test_labels = createDataset()

# Model


## Create the model

Here we define the model architecture using keras's sequential model.

In [ ]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([
                                     
        tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(24, 32, 1)),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),
        
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),
        
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=128, activation=tf.nn.relu),

        tf.keras.layers.Dense(units=128, activation=tf.nn.relu),

        tf.keras.layers.Dense(units=64, activation=tf.nn.relu),

        tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid),
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()

# Initialize the model by passing some data through
# cnn_model.predict(train_images[[0]])

# Compile the model
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the summary of the layers in the model.
print(cnn_model.summary())

## Train the model

You can load a saved / pre-trained model by specifying the path to the model in the below cell.

In [ ]:
cnn_model = tf.keras.models.load_model("models/MLX90640-Human-Detection")

# Print the summary of the layers in the model.
print(cnn_model.summary())

Train the model for a fixed number of epochs.

In [ ]:
cnn_model.fit(train_images, train_labels, epochs=5)

Or train the model until the testing accuracy stops improving. This way can help prevent overfitting of the model to the training dataset.

In [ ]:
acc = 0.5
val_acc = 0.5
last_acc = 0
last_val_acc = 0
while val_acc >= last_val_acc and acc > last_acc:
  h = cnn_model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels)).history
  last_acc = acc
  last_val_acc = val_acc
  acc = h['accuracy'][-1]
  val_acc = h['val_accuracy'][-1]
  
print(round(val_acc,3))

Save the trained model for future use by specifying the path to the location to save it at in the below cell.

In [ ]:
tf.keras.models.save_model(cnn_model, "models/MLX90640-2")

## Test the model


You can test the accuracy of the model on the test set by running the below cell.

In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)